# Idealista API

Script for sending an API query to Idealista and storing the received data to the CSV file 'data.csv'. If the file doesn't exist, it will be created.

API credentials are stored in 'credentials.py' file in format: <br>
*apikey = 'xxxxxxxx'* <br>
*secret = 'xxxxxxx'* <br>


In [1]:
import pandas as pd
import os.path
import json
import urllib
import requests as rq
import datetime as dt
import credentials #python script with stored apikey and secret (sent by idealista)

pd.set_option('display.max_columns', None)


apikey= credentials.apikey
secret= credentials.secret 

def get_oauth_token(key, secret):
    # Get OAuth token from Idealista
    oauth_url = "https://api.idealista.com/oauth/token"
    parameters = {"grant_type":"client_credentials"}
    r = rq.post(oauth_url,
                      auth=rq.auth.HTTPBasicAuth(key, secret),
                      data=parameters)
    received_token = json.loads(r.text)
    return received_token["access_token"]

def search_api(token, url):    
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    df = pd.DataFrame.from_dict(result['elementList'])
    return df


In [2]:
country = 'es' # possible values: es, it, pt
locale = 'es'  #possible values: es, it, pt, en, ca
language = 'es' #
max_items = '50'
operation = 'sale' 
property_type = 'homes'
order = 'priceDown' 
center = "41.3881,2.1657"
distance = '7000'
sort = 'desc'
countryHouse = 'false'
chalet = 'false'
maxPrice = '410000'
sinceDate = 'W'
#bankOffer = 'false'

# maximum number of queries to perform (maximum number of fetched properties 
# per a query is 'max_items')
limit = 50

df_tot = pd.DataFrame()

for i in range(1,limit):
    url = ('https://api.idealista.com/3.5/'+country+'/search?operation='+operation+#"&locale="+locale+
           '&maxItems='+max_items+
           '&order='+order+
           '&center='+center+
           '&distance='+distance+
           '&propertyType='+property_type+
           '&sort='+sort+ 
           '&numPage=%s'+
           '&countryHouse='+countryHouse+ 
           '&chalet='+chalet+ 
           '&maxPrice='+maxPrice+ 
           '&sinceDate='+sinceDate+
           '&language='+language) %(i)  
    
    df = search_api(get_oauth_token(apikey, secret), url)
    df_tot = pd.concat([df_tot,df])
    if df.shape[0]<limit:
        print('All properties downloaded')
        break

df_tot = df_tot.reset_index()

Broke ahead


In [3]:
df_tot["Week"] = dt.date.today().isocalendar()[1]

if os.path.isfile('./data/data.csv'):
    # append csv
    df_stored = pd.read_csv(('./data/data.csv'))
    df_to_store = pd.concat([df_stored, df_tot])
    df_to_store.to_csv("./data/data.csv",index=False)
    
else:
    # create csv
    print('Creating new file with data')
    df_tot.to_csv('./data/data.csv',index=False)
